# The Battle of Neigbourhoods PART 2

## This project is to be submitted as the Coursera Capstone assignment 

In [1]:
import requests 
import pandas as pd 
import numpy as np 
import random 
print('Done')

Done


In [ ]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: - 

In [ ]:

Sales_NYC.shape
Sales_NYC.columns

In [ ]:
SalesNY=Sales_NYC[["BOROUGH","Hood", "BUILDING CLASS CATEGORY","ZIP CODE","TOTAL UNITS","LAND SQUARE FEET","GROSS SQUARE FEET","YEAR BUILT","SALE DATE"," SALE PRICE "]]

SalesNY.rename(columns={" SALE PRICE ": "SALE PRICE"}, inplace=True)
SalesNY.head()

In [ ]:
SalesNY.dtypes

In [ ]:
SalesNY[['SALE PRICE','LAND SQUARE FEET','GROSS SQUARE FEET']]

In [ ]:
SalesNY['SALE PRICE']=SalesNY['SALE PRICE'].str.replace(',','')
SalesNY["SALE PRICE"]=SalesNY["SALE PRICE"].astype(int)

In [ ]:
SalesNY.dtypes

In [ ]:
SalesNY['LAND SQUARE FEET']=SalesNY['LAND SQUARE FEET'].str.replace(',','')
SalesNY['GROSS SQUARE FEET']=SalesNY['GROSS SQUARE FEET'].str.replace(',','')

SalesNY["LAND SQUARE FEET"]=SalesNY["LAND SQUARE FEET"].astype(float)
SalesNY["GROSS SQUARE FEET"]=SalesNY["GROSS SQUARE FEET"].astype(float)

In [ ]:
missing_data = SalesNY.isnull()
missing_data.head(5)

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

In [ ]:
Total_entries=SalesNY.groupby(['BUILDING CLASS CATEGORY'],as_index=False).count()
Total_entries

In [ ]:
Total_entries.sort_values(by='Hood', ascending=True, inplace=True)
Total_entries

In [ ]:
Top_entries=Total_entries[Total_entries.Hood > 110]
Top_entries.set_index('BUILDING CLASS CATEGORY',inplace=True)

import matplotlib.pyplot as plt

Top_entries['BOROUGH'].plot(kind='bar', figsize=(10, 6))
plt.ylabel('Total Data Entries')
plt.title('Total Data Entries by real estate type')


plt.show()

In [ ]:
dfNY=SalesNY[SalesNY['BUILDING CLASS CATEGORY'].isin(Top_entries.index)]
dfNY.groupby(['BUILDING CLASS CATEGORY'],as_index=False).count()

In [ ]:
TrueSale=dfNY[dfNY["SALE PRICE"]>100001]
TrueSale.reset_index(drop=True)

In [ ]:
DropClass=['21 OFFICE BUILDINGS','43 CONDO OFFICE BUILDINGS','47 CONDO NON-BUSINESS STORAGE','22 STORE BUILDINGS']
TrueSale=TrueSale[~TrueSale['BUILDING CLASS CATEGORY'].isin(DropClass)]

In [ ]:
TrueSale.groupby(['BUILDING CLASS CATEGORY'],as_index=False).count()

In [ ]:
TrueSale.groupby(['BUILDING CLASS CATEGORY','Hood'],as_index=False).median()

In [ ]:
MedianSales=TrueSale.groupby(['BUILDING CLASS CATEGORY'],as_index=False).median()
MedianSales


In [ ]:
import seaborn as sns

ax=sns.boxplot(x=TrueSale['BUILDING CLASS CATEGORY'], y=TrueSale['SALE PRICE']/1000000 )
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set(xlabel='Residential Type', ylabel='Sale price (M$)')
ax.set(ylim=(0, 100))
plt.title('Sale price in M$ by residential type')
#plt.show()

In [ ]:
TrueSale.shape

In [ ]:
TrueSale["TOTAL UNITS"].replace(np.nan, 1, inplace=True)

TrueSale['PricePerUnit']=TrueSale['SALE PRICE']/(TrueSale['TOTAL UNITS']*1000000)
TrueSale['SFperUnit']=TrueSale['GROSS SQUARE FEET']/(TrueSale['TOTAL UNITS'])
TrueSale

In [ ]:
import seaborn as sns

ax=sns.boxplot(x=TrueSale['BUILDING CLASS CATEGORY'], y=TrueSale['PricePerUnit'])
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set(xlabel='Residential Type', ylabel='Sale price (M$)')
ax.set(ylim=(0, 20))
plt.title('Sale price in M$ by residential type')
#plt.show()

In [ ]:
MedianSalesPerUnit=TrueSale[['BUILDING CLASS CATEGORY','PricePerUnit']].groupby(['BUILDING CLASS CATEGORY'],as_index=False).median()
MedianSalesPerUnit

In [ ]:
ax=sns.lmplot(x="SFperUnit", y="PricePerUnit", hue="BUILDING CLASS CATEGORY", data=TrueSale[TrueSale["GROSS SQUARE FEET"]>0]);
ax.set(xlabel='Gross square feet per unit', ylabel='Price per unit (M$)')
ax.set(ylim=(0, 40))
plt.title('Square feet vs. Price per unit')

In [ ]:
TrueSale[TrueSale["GROSS SQUARE FEET"]>0][['SFperUnit', 'PricePerUnit']].corr()

In [ ]:
lm = LinearRegression()
X = TrueSale[TrueSale["GROSS SQUARE FEET"]>0][['SFperUnit']]
Y = TrueSale[TrueSale["GROSS SQUARE FEET"]>0][['PricePerUnit']]

In [ ]:
lm.fit(X,Y)
print("PricePerUnit=Slope*SFPerUnit + Intercept")
print("Intercept:",lm.intercept_)
print("Slope:",lm.coef_)

##  Neighbourhood analysis

In [ ]:
DropClass1=['01 ONE FAMILY DWELLINGS','02 TWO FAMILY DWELLINGS']
TrueSale=TrueSale[~TrueSale['BUILDING CLASS CATEGORY'].isin(DropClass1)]
TrueSale

In [ ]:
HoodMedianSale=TrueSale[['Hood','PricePerUnit']].groupby(['Hood'],as_index=False).median()
HoodMedianSale

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [ ]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

In [ ]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [ ]:
neighborhoods_data = newyork_data['features']

In [ ]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

##Then let's loop through the data and fill the dataframe one row at a time.


for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [ ]:
neighborhoods.head()

In [ ]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

In [ ]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

In [ ]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [ ]:
HoodMedianSale = HoodMedianSale.merge(manhattan_data, left_on="Hood", right_on="Neighborhood", how="outer", suffixes=("_tableA", "_tableB"))

In [ ]:
HoodMedianSale

In [ ]:
HoodMedianSale.dropna(subset=["Hood"], axis=0,inplace=True)
HoodMedianSale

In [ ]:
bins = np.linspace(0, 3, 5)
digitized = np.digitize(HoodMedianSale['PricePerUnit'], bins)
HoodMedianSale['Cluster']=digitized
## Number of unique clusters 
k_cluster=len(np.unique(digitized))

HoodMedianSale

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(k_cluster)
ys = [i + x + (i*x)**2 for i in range(k_cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster,medianCost in zip(HoodMedianSale['Latitude'], HoodMedianSale['Longitude'], HoodMedianSale['Hood'], HoodMedianSale['Cluster'],HoodMedianSale['PricePerUnit']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster)+' Median Residential Unit Cost (M$)' +str(medianCost), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
CLIENT_ID = 'VRX22JOOMQ33YSFLEBWV4C0V1T2HTGBDKTTVTHIWGJHTD44O' # your Foursquare ID
CLIENT_SECRET = '1FQWLS5JKJKLVB51LSZEAM4KJYTIRJAU334QIJG1I1O102RV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 300 #limit of number of venues returned by Foursquare API
radius= 500 ## Radius of venues from address

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
manhattan_venues = getNearbyVenues(names=HoodMedianSale['Hood'],
                                   latitudes=HoodMedianSale['Latitude'],
                                   longitudes=HoodMedianSale['Longitude']
                                  )

In [ ]:
print(manhattan_venues.shape)
manhattan_venues.head()

In [ ]:
Total_venues=manhattan_venues.groupby('Neighborhood').count()
Total_venues

In [ ]:
Total_venues.shape

In [ ]:
MedianSellVenue = HoodMedianSale.merge(Total_venues["Venue"], left_on="Hood", right_on="Neighborhood", how="outer", suffixes=("_tableA", "_tableB"))
MedianSellVenue

In [ ]:
ax=sns.lmplot(x="Venue", y="PricePerUnit", data=MedianSellVenue);
ax.set(xlabel='Total venues in area', ylabel='Price per unit (M$)')
ax.set(ylim=(0, 5))
plt.title('Total venues in the area vs. price per residential unit')

In [ ]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

In [ ]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

In [ ]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
kVenue = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kVenue, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

In [ ]:
manhattan_merged.dropna(subset=["Cluster Labels"], axis=0,inplace=True)
manhattan_merged

In [ ]:
MedianSellVenue = MedianSellVenue.merge(manhattan_merged.iloc[:,[1,4,5,6,7,8,9]], left_on="Hood", right_on="Neighborhood", how="outer", suffixes=("_1", "_2"))
#manhattan_merged.iloc[:,[1,4,5,6,7,8,9]]

                                         
MedianSellVenue

In [ ]:
MedianSellVenue=MedianSellVenue.rename(columns={'Cluster': 'Price Cluster', 'Cluster Labels': 'Venue Cluster'})
MedianSellVenue

In [ ]:
ax=sns.lmplot(x="Venue", y="PricePerUnit", hue="Venue Cluster", data=MedianSellVenue);

ax.set(xlabel='Total venues in area', ylabel='Price per unit (M$)')
ax.set(ylim=(0, 5))
plt.title('Total venues in the area vs. price per residential unit - Clustered by neighborhood with most common venues types')

In [ ]:
MedianSellVenue.loc[MedianSellVenue['Venue Cluster'] == 0]

In [ ]:
MedianSellVenue.groupby(['Venue Cluster'],as_index=False).mean()

In [ ]:
MedianSellVenue.loc[MedianSellVenue['Venue Cluster'] == 1]

In [ ]:
MedianSellVenue.loc[MedianSellVenue['Venue Cluster'] == 2]

In [ ]:
MedianSellVenue.loc[MedianSellVenue['Venue Cluster'] == 3]

In [ ]:
MedianSellVenue.loc[MedianSellVenue['Venue Cluster'] == 4]